In [71]:
from pyspark.sql.types import *
import pandas as pd
import pyspark 
import os 
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, udf, rank, asc, sum as spark_sum
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.window import Window

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


In [72]:
# Start a Spark session
spark = SparkSession.builder.appName("s33ding").getOrCreate()

# Read the Parquet file into a DataFrame
df = spark.read.parquet("dataset/enem.parquet")
# Select the relevant columns
selected_cols = ["NOTA_CH_CIENCIAS_HUMANAS", "NOTA_LC_LINGUAGENS_E_CODIGOS", "NOTA_MT_MATEMATICA", "NOTA_REDACAO","NOTA_CN_CIENCIAS_DA_NATUREZA"]
df_selected = df.select(selected_cols)
df_selected.show(3)

+------------------------+----------------------------+------------------+------------+----------------------------+
|NOTA_CH_CIENCIAS_HUMANAS|NOTA_LC_LINGUAGENS_E_CODIGOS|NOTA_MT_MATEMATICA|NOTA_REDACAO|NOTA_CN_CIENCIAS_DA_NATUREZA|
+------------------------+----------------------------+------------------+------------+----------------------------+
|                   478.4|                       426.8|             351.2|       380.0|                       411.3|
|                   459.3|                       571.7|             467.0|       900.0|                       440.0|
|                   435.7|                       568.2|             414.7|       960.0|                       607.1|
+------------------------+----------------------------+------------------+------------+----------------------------+
only showing top 3 rows



In [73]:
from pyspark.sql.functions import monotonically_increasing_id

# Split the dataset into training, validation, and testing sets
train_data, validation_data, test_data = df_selected.randomSplit([0.6, 0.2, 0.2], seed=42)

# Prepare the feature vector and the target column for the training, validation, and testing sets
assembler = VectorAssembler(inputCols=["NOTA_LC_LINGUAGENS_E_CODIGOS", "NOTA_MT_MATEMATICA", "NOTA_REDACAO"], outputCol="features")

# Transform the training data and add a primary key column
train_data = assembler.transform(train_data).select("features", "NOTA_CN_CIENCIAS_DA_NATUREZA")
train_data = train_data.withColumn("pk", monotonically_increasing_id())

# Transform the validation data and add a primary key column
validation_data = assembler.transform(validation_data).select("features", "NOTA_CN_CIENCIAS_DA_NATUREZA")
validation_data = validation_data.withColumn("pk", monotonically_increasing_id())

# Transform the testing data and add a primary key column
test_data = assembler.transform(test_data).select("features", "NOTA_CN_CIENCIAS_DA_NATUREZA")
test_data = test_data.withColumn("pk", monotonically_increasing_id())

# Train the machine learning models using the training set
# Linear Regression
lr = LinearRegression(labelCol="NOTA_CN_CIENCIAS_DA_NATUREZA")
lr_model = lr.fit(train_data)

# Decision Tree Regression
dt = DecisionTreeRegressor(labelCol="NOTA_CN_CIENCIAS_DA_NATUREZA")
dt_model = dt.fit(train_data)

# Random Forest Regression
rf = RandomForestRegressor(labelCol="NOTA_CN_CIENCIAS_DA_NATUREZA")
rf_model = rf.fit(train_data)

# Make predictions using the trained models on the validation set
lr_predictions = lr_model.transform(validation_data)
dt_predictions = dt_model.transform(validation_data)
rf_predictions = rf_model.transform(validation_data)


23/06/14 01:07:19 WARN Instrumentation: [045749b4] regParam is zero, which might cause numerical instability and overfitting.


In [79]:
from pyspark.sql.functions import col

# Create temporary views for the predictions
lr_predictions.createOrReplaceTempView("lr_predictions")
dt_predictions.createOrReplaceTempView("dt_predictions")
rf_predictions.createOrReplaceTempView("rf_predictions")

# Join the predictions using the primary key (pk) column
joined_predictions = spark.sql("""
    SELECT lr_predictions.features, lr_predictions.prediction AS lr_prediction,
           dt_predictions.prediction AS dt_prediction, rf_predictions.prediction AS rf_prediction,
           lr_predictions.NOTA_CN_CIENCIAS_DA_NATUREZA
    FROM lr_predictions
    JOIN dt_predictions ON lr_predictions.pk = dt_predictions.pk
    JOIN rf_predictions ON lr_predictions.pk = rf_predictions.pk
    ORDER BY lr_predictions.pk
""")


# Get a sample from the joined_predictions DataFrame
sample_joined_predictions = joined_predictions.sample(fraction=0.1, seed=42)

# Save the sample as a Parquet file
sample_joined_predictions.write.mode('overwrite').parquet('data_for_dashboards/models/joined_predictions.parquet')

# Show the joined predictions
joined_predictions.show()


+--------------------+------------------+------------------+------------------+----------------------------+
|            features|     lr_prediction|     dt_prediction|     rf_prediction|NOTA_CN_CIENCIAS_DA_NATUREZA|
+--------------------+------------------+------------------+------------------+----------------------------+
| [413.0,444.0,360.0]| 417.1890685808009| 423.1020666217345| 429.0939459227814|                       381.8|
|[345.399993896484...| 381.6734296910864| 423.1020666217345| 425.0362786947635|                       381.4|
|[326.700012207031...|387.33017447876017| 423.1020666217345| 433.6279647120011|                       434.2|
|[385.200012207031...| 383.8776007102846| 423.1020666217345|424.60054080470434|                       448.3|
|[334.600006103515...|408.02071087368336|436.36443385870047| 445.1555965113309|                       352.7|
|[434.200012207031...|460.03156244267313|462.94221956370393| 454.7604784088786|                       422.4|
|[498.0,458.299987.

In [82]:
from pyspark.ml.evaluation import RegressionEvaluator

# Define the evaluator
evaluator = RegressionEvaluator(labelCol="NOTA_CN_CIENCIAS_DA_NATUREZA", predictionCol="prediction")

# Calculate MSE for linear regression
lr_mse = evaluator.evaluate(lr_predictions, {evaluator.metricName: "mse"})

# Calculate MSE for decision tree regression
dt_mse = evaluator.evaluate(dt_predictions, {evaluator.metricName: "mse"})

# Calculate MSE for random forest regression
rf_mse = evaluator.evaluate(rf_predictions, {evaluator.metricName: "mse"})

# Calculate MAE for linear regression
lr_mae = evaluator.evaluate(lr_predictions, {evaluator.metricName: "mae"})

# Calculate MAE for decision tree regression
dt_mae = evaluator.evaluate(dt_predictions, {evaluator.metricName: "mae"})

# Calculate MAE for random forest regression
rf_mae = evaluator.evaluate(rf_predictions, {evaluator.metricName: "mae"})

# Calculate MSE and MAE for each model
metrics = [("Linear Regression - MSE", lr_mse),
           ("Decision Tree Regression - MSE", dt_mse),
           ("Random Forest Regression - MSE", rf_mse),
           ("Linear Regression - MAE", lr_mae),
           ("Decision Tree Regression - MAE", dt_mae),
           ("Random Forest Regression - MAE", rf_mae)]

# Create the DataFrame with "Type" column
df_comparing_models = spark.createDataFrame(metrics, ["Model", "Metric"]).withColumn("Type", udf(lambda model: "MSE" if "MSE" in model else "MAE", StringType())("Model"))

# Create a window specification to partition by the type and order by the metric in ascending order
window_spec = Window.partitionBy("Type").orderBy(asc("Metric"))

# Add a "Best" column to determine the best model for each type
df_comparing_models = df_comparing_models.withColumn("Best", rank().over(window_spec) == 1)

# Show the DataFrame
df_comparing_models.show(truncate=False)
tmp = df_comparing_models.toPandas()
tmp.to_parquet('data_for_dashboards/eda/models')



+------------------------------+------------------+----+-----+
|Model                         |Metric            |Type|Best |
+------------------------------+------------------+----+-----+
|Random Forest Regression - MAE|42.46582183771729 |MAE |true |
|Linear Regression - MAE       |42.53800192500141 |MAE |false|
|Decision Tree Regression - MAE|42.59457338590594 |MAE |false|
|Random Forest Regression - MSE|2767.006748070056 |MSE |true |
|Linear Regression - MSE       |2817.5776266367934|MSE |false|
|Decision Tree Regression - MSE|2822.5567248988423|MSE |false|
+------------------------------+------------------+----+-----+



In [83]:
# Specify the path to save the model
model_path = "models/nota_ch_ciencias_humanas/random_forest"

# Save the Linear Regression model
rf_model.save(model_path)